# Initiate

In [ ]:
print('Initiate in process ...')

from Import_pakages import *
import Import_functions as Import_functions
import Conversion_functions as Conversion_functions
import Vis_functions as Vis_functions

from pathlib import Path
output_filepath = '/Users/joe/Downloads/'  # The file path for the output files

%matplotlib widget
%load_ext autoreload
%autoreload 2

print('Initiate complete')

# Some  clarification

The three axes of the 3D data matrix are set in the order of [Emission angle (Y), Binding energy (eb), Photon energy (hv)]

Symbols:
    
* Y: Emission Angle (Deg)
* eb: Binding Energy (eV)
* hv: Photon Energy (eV)
* kp: $k_\parallel$ (Å$^{-1})$
* kz: $k_z$ (Å$^{-1})$
* E_work: Work Function, defaut is set to 4.42 (eV)
* T: Temperature (K)
* v0: Inner Potential, defaut is set to 0 (eV)

# Parameters Input + Normalization to data1

## Improt from .h5 file

In [ ]:
filepath = '/Users/joe/Documents/EPFL/LQM/EuIn2P2/Data/EuIn2P2_ARPES/SLS_2022_10_17/Sample 1/hv_scan_0001.h5'

sample_name = ['20221017', 'EuIn2P2']

data0, axes, hv, E_work, T = Import_functions.import_h5py(filepath)

if len(axes) == 3:
    # Assuming shape is (y_len, x_len, hv_len)
    hv = axes[2]
else:
    data0 = data0[:, :, np.newaxis] 

# 4. Generate Coordinates
Y = axes[1]
eb = -axes[0]
    
# 5. Efficient Normalization
# Compute mean along spatial axes (0 and 1)
# This is much faster if done before swapping axes
in_sum = np.mean(data0, axis=(0, 1))
data1 = data0 / in_sum[np.newaxis, np.newaxis, :]
data0 = np.transpose(data0, (1, 0, 2))
data1 = np.transpose(data1, (1, 0, 2))

print('Importing completed')

# Binding Energy Raw Plot

In [ ]:
plt.close()
fig, ax = Vis_functions.slice_3D(data=data1, axes=[Y, eb, hv], sample_name=sample_name, output_filepath=output_filepath)
plt.show()

In [ ]:
Y = Y+0.5

# Fermi Edge Alignment data2 ($Y$, $E_b$, hv)

## Fermi Egde Fitting

### Integration over emission angles

In [ ]:
# Binding Energy Range for Fermi Edge Fitting
eb_range = [-0.15, 0.2]

# Matrix for Intensity data integrated over angles
data1_angle_sum = np.sum(data1, axis=0)

plt.close()
Vis_functions.EDC_angle_sum(data=data1_angle_sum, eb=eb, z=hv, eb_range=eb_range, sample_name=sample_name, output_filepath=output_filepath)
plt.show()

### Fermi-Dirac Model Fitting

In [ ]:
fit_range = [0.15, -0.15] # The data range in index of binding energy for fitting

fermi_fit_results = Conversion_functions.Fermi_level_fit_fd(data1_angle_sum, fit_range, eb, hv, T)

Fermi_fit_dict = {'offset': fermi_fit_results[:, 0], 
                  'Temp': fermi_fit_results[:, 1],
                  'fermi_edge_amplitude': fermi_fit_results[:, 2],
                  'fermi_edge_const_bkg': fermi_fit_results[:, 3],
                  'fermi_edge_lin_bkg': fermi_fit_results[:, 4] }

print('Fitting Completed')
print('R-squared:')
print(fermi_fit_results[:,-1] )

In [ ]:
plt.close()
Vis_functions.Fermi_edge_fit_plot(data=data1_angle_sum, eb=eb, hv=hv, fit_range=fit_range,
                                               fermi_edge_center=fermi_fit_results[:, 0], fermi_edge_T= fermi_fit_results[:, 1],
                                               fermi_edge_amplitude=fermi_fit_results[:, 2], fermi_edge_const_bkg=fermi_fit_results[:, 3], fermi_edge_lin_bkg=fermi_fit_results[:, 4],
                                               sample_name=sample_name, output_filepath=output_filepath)
plt.show()

### Affine_broadened_fd Model Fitting

In [ ]:
fit_range = [0.15, -0.15] # The data range in index of binding energy for fitting

fermi_fit_results = Conversion_functions.Fermi_level_fit_ABfd(data1_angle_sum, fit_range, eb, hv, T, center=[0, -0.01, 0.04])

Fermi_fit_dict = {'offset': fermi_fit_results[:, 0], 
                  'Temp': fermi_fit_results[:, 1],
                  'fermi_edge_conv_width': fermi_fit_results[:, 2],
                  'fermi_edge_const_bkg': fermi_fit_results[:, 3],
                  'fermi_edge_lin_bkg': fermi_fit_results[:, 4],
                  'BG_offset': fermi_fit_results[:, 5] }

print('Fitting Completed')
print('R-squared:')
print(fermi_fit_results[:,-1] )
fermi_edge_center_index = find_value_index(eb, fermi_fit_results[:, 0])

In [ ]:
fermi_edge_center_index = find_value_index(eb, fermi_fit_results[:, 0])

plt.close()
fig, ax1, ax2, ax3= Vis_functions.Affine_broadened_Fermi_edge_fit_plot(data=data1_angle_sum, eb=eb, hv=hv, fit_range=fit_range,
                                               fermi_edge_center=fermi_fit_results[:, 0], fermi_edge_T= fermi_fit_results[:, 1], fermi_edge_conv_width=fermi_fit_results[:, 2], 
                                               fermi_edge_const_bkg=fermi_fit_results[:, 3], fermi_edge_lin_bkg=fermi_fit_results[:, 4], fermi_edge_offset=fermi_fit_results[:, 5], 
                                               sample_name=sample_name, output_filepath=output_filepath)
plt.show()

### Fit Offests with Linear function of $h\nu$

In [ ]:
fermi_edge_center_index = find_value_index(eb, fermi_fit_results[:, 0])

#Linear fitting of Fermi edge center vs hv for alignment
def linear_func(x, slope, c): 
    return slope * x + c
mod = lmfit.Model(linear_func)
params = mod.make_params(slope=1, c=0)
out = mod.fit(fermi_fit_results[:, 0], params, x=hv, method='nelder')
print(out.fit_report())
fermi_edge_center_index = find_value_index(eb, linear_func(hv, out.params['slope'].value, out.params['c'].value))

plt.close()
fig, ax = plt.subplots(1, 1, figsize=(5,2), layout='constrained')

ax.plot(hv, fermi_fit_results[:, 0], 'o', label='Data', markersize=4)
ax.plot(hv, linear_func(hv, out.params['slope'].value, out.params['c'].value), '-', label='Fit', markersize=4)
plt.show()

## Alignment & output

In [ ]:
print('Alignment in process ...')

Eb_0_index = np.abs(eb - 0).argmin()
# 1. Pre-calculate shifts for each k
shifts = np.round(Eb_0_index - fermi_edge_center_index).astype(int)
# 2. Vectorized approach
data2 = np.zeros_like(data1)
for k, shift in enumerate(shifts):
    # Process the entire [Y, eb] slice for this 'k' at once
    if shift > 0:
        data2[:, shift:, k] = data1[:, :-shift, k]
    elif shift < 0:
        data2[:, :shift, k] = data1[:, -shift:, k]
    else:
        data2[:, :, k] = data1[:, :, k]
print('Alignment completed')

matlab_mat = {"data": data2, 'Y': Y, 'Eb': eb, 'hv': hv}
matlab_mat =  matlab_mat | Fermi_fit_dict

file_path = Path(output_filepath+sample_name[1]+'_Y_Eb_hv.mat')
if not file_path.exists():
    savemat(file_path, matlab_mat)
    print(f"Saved to {file_path}")
else:
    print( '\033[31m' + f"File {file_path} already exists. Skipping save." + '\033[0m' )

## Alignment check

In [ ]:
plt.close()
fig, ax = Vis_functions.slice_3D(data=data2, axes=[Y, eb, hv], sample_name=sample_name, output_filepath=output_filepath)
plt.show()

### EDCs averaging over emission angles

In [ ]:
data2_angle_sum = np.sum(data2, axis=0)
        
eb_range = [-0.1, 0.15]

plt.close()
Vis_functions.EDC_angle_sum(data=data2_angle_sum, eb=eb, z=hv, eb_range=eb_range, sample_name=sample_name, output_filepath=output_filepath)
plt.show()

# Convert $Y$ to $k_p$ data3 ($k_p$, $E_b$, $hv$)

In [ ]:
kp_res_factor = 1
kp, data3 = Conversion_functions.convert_angle_to_kp(
    data=data2, 
    angle_axis=Y, 
    eb_axis=eb, 
    hv_axis=hv, 
    work_function=E_work,
    k_res_factor=kp_res_factor
)

matlab_mat = {"data": data3, 'kp': kp, 'Eb': eb, 'hv': hv, 'E_work': E_work, 'kp_res_factor': kp_res_factor}
matlab_mat = matlab_mat | Fermi_fit_dict

file_path = Path(output_filepath+sample_name[1]+'_kp_Eb_hv_interp.mat')
if not file_path.exists():
    savemat(file_path, matlab_mat)
    print(f"Saved to {file_path}")
else:
    print( '\033[31m' + f"File {file_path} already exists. Skipping save." + '\033[0m' )

## Visualization

In [ ]:
plt.close()
fig, ax = Vis_functions.slice_3D(data=data3, axes=[kp, eb, hv], sample_name=sample_name, output_filepath=output_filepath)
plt.show()

# Convert $h\nu$ to $k_z$ (data4 or data3)

## Find suitable inner potential v0

In [ ]:
#BZ_length = 1.72    # 1 BZ period for SnTe
#BZ_length = 1.76    # 1 BZ period for GeTe
BZ_length = 0.357    # 1 BZ period for EuIn2P2

plt.close()
Vis_functions.Tune_V0(data=data3, x=kp, y=eb, hv=hv, BZ_len=BZ_length, x_name='kp', z_name='kz', sample_name=sample_name, output_filepath=output_filepath, E_work=E_work)
plt.show()

## Conversion & output

### Interpolate Conversion data4 ($k_p$, $E_b$, $k_z$)

In [ ]:
v0 = 10  # Inner potential
kz_res_factor = 1

kz, data4 = Conversion_functions.convert_hv_to_kz(
    data=data3,
    k_para_axis=kp, 
    eb_axis=eb, 
    hv_axis=hv, 
    V0=v0, 
    work_function=E_work,
    kz_res_factor=kz_res_factor
)

matlab_mat = {"data": data4, 'kp': kp, 'Eb': eb, 'kz': kz, 'V0':v0, 'E_work': E_work, 'kp_res_factor': kp_res_factor, 'kz_res_factor': kz_res_factor}
matlab_mat = matlab_mat | Fermi_fit_dict

file_path = Path(output_filepath+sample_name[1]+'_kp_Eb_kz_interp.mat')
if not file_path.exists():
    savemat(file_path, matlab_mat)
    print(f"Saved to {file_path}")
else:
    print( '\033[31m' + f"File {file_path} already exists. Skipping save." + '\033[0m' )

### Relabel -- Convert hv to $k_z$ at $k_p = 0$ and $E_b = 0$ (Only change the axis lable value) data3 ($k_p$, $E_b$, $k_z$)

In [ ]:
v0 = 10  # Inner potential
kz = np.array([])

for k in range(len(hv)):
    kz = np.append(kz,   1/hbar * np.sqrt( 2*me* (hv[k] - 0 - E_work + v0) )   )
    

matlab_mat = {"data": data3, 'kp': kp, 'Eb': eb, 'kz': kz, 'V0':v0, 'E_work': E_work, 'kp_res_factor': kp_res_factor}
matlab_mat = matlab_mat | Fermi_fit_dict

savemat(output_filepath+sample_name[1]+'_kp_Eb_kz.mat', matlab_mat)
print('Data3 saved to "' + output_filepath+sample_name[1]+'_kp_Eb_kz.mat"')

file_path = Path(output_filepath+sample_name[1]+'_kp_Eb_kz_relabel.mat')
if not file_path.exists():
    savemat(file_path, matlab_mat)
    print(f"Saved to {file_path}")
else:
    print( '\033[31m' + f"File {file_path} already exists. Skipping save." + '\033[0m' )

## Visualization

In [ ]:
plt.close()
fig, ax = Vis_functions.slice_3D(data=data4, axes=[kp, eb, kz], sample_name=sample_name, output_filepath=output_filepath)
plt.show()